# 1. Let's import necessary libraries

In [1]:
import pyemu
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import matplotlib.pyplot as plt
import psutil
import shutil
import numpy as np
import sys
import flopy

## work with the latest version of swatmf

In [2]:
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(1, path)
from swatmf import swatmf_pst_utils

# 2. Set up and write swatmf.con file

In [ ]:
swatmf_pst_utils.create_swatmf_con?

In [4]:
# working directory
swatmf_model = "D:/test/middle_bosque_1000/SWAT-MODFLOW"
swat_model = "D:/test/middle_bosque_1000/SWAT"

In [7]:
# calibration period
sim_start = '1/1/1985'
warmup = 0
cal_start = '1/1/1985'
cal_end = '12/31/1985'

# time step
time_step = 'day'

# locations (what our targets)
subs = [58]
grids = [501]

In [8]:
swatmf_pst_utils.create_swatmf_con(
    swatmf_model, sim_start, warmup, cal_start, cal_end, subs=subs, grids=grids)

,names,vals
0,wd,D:/test/middle_bosque_1000/SWAT-MODFLOW
1,sim_start,1/1/1985
2,warm-up,0
3,cal_start,1/1/1985
4,cal_end,12/31/1985
5,subs,[58]
6,grids,[501]
7,riv_parm,n
8,baseflow,n
9,time_step,day


## 2.1 PEST initial setup

In [9]:
# copy all necessary files (exes) to your working direcotry
swatmf_pst_utils.init_setup(swatmf_model, swat_model)

100%|████████████████████████████████████████████████████████████████████████████| 12300/12300 [02:17<00:00, 89.64it/s]

 Creating 'backup' folder ... passed
 Creating 'echo' folder ... passed
 Creating 'sufi2.in' folder ... passed


In [10]:
# check MODFLOW model
mname = "modflow.mfn"
m = flopy.modflow.Modflow.load(mname,model_ws=swatmf_model)
m.check()


modflow MODEL DATA VALIDATION SUMMARY:
  8 Errors:
    DIS package: thin cells (less than checker threshold of 1.0)
    RIV package: rbot below cell bottom
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    UPW package: zero or negative horizontal hydraulic conductivity values
    UPW package: zero or negative vertical hydraulic conductivity values
    UPW package: negative horizontal anisotropy values
    UPW package: horizontal hydraulic conductivity values below che

# 3. Build template files

In [11]:
os.chdir(swatmf_model)

## 3.1 MODFLOW pval

In [12]:
# pval file
pval_file = 'mf_1000.pval'

In [13]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
gw_par

,parnme,parval1,tpl
parnme,,,
hk01,hk01,1.00,~ hk01 ~
hk02,hk02,1.00,~ hk02 ~
sy01,sy01,0.01,~ sy01 ~
sy02,sy02,0.01,~ sy02 ~


## 3.2 SWAT model.in file

In [14]:
# model.in file used
sw_par = swatmf_pst_utils.model_in_to_template_file()
sw_par

,parnme,parval1,tpl
parnme,,,
r__CN2.mgt,r__CN2.mgt,0.001,~ CN2 ~
r__SOL_AWC().sol,r__SOL_AWC().sol,0.001,~ SOL_AWC() ~
v__ESCO.hru,v__ESCO.hru,0.001,~ ESCO ~


# 4. Build instruction files

### Let's do initial run!

## 4.1 Streamflow (SWAT)

In [15]:
# extract daily stream discharge
swatmf_pst_utils.extract_day_stf(subs, sim_start, warmup, cal_start, cal_end)

stf_058.txt file has been created...
Finished ...


## 4.2 match it with stf_obd file (SWAT)

In [16]:
swatmf_pst_utils.stf_obd_to_ins('stf_058.txt', 'rch058',cal_start, cal_end)

stf_058.txt.ins file has been created...


date
1985-01-01    l1 w !rch058_19850101!
1985-01-02    l1 w !rch058_19850102!
1985-01-03    l1 w !rch058_19850103!
1985-01-04    l1 w !rch058_19850104!
1985-01-05    l1 w !rch058_19850105!
                       ...          
1985-12-27                        l1
1985-12-28                        l1
1985-12-29                        l1
1985-12-30                        l1
1985-12-31                        l1
Freq: D, Name: rch058_ins, Length: 365, dtype: object

## 4.3 Depth to watertable (MODFLOW) 

In [17]:
mf_obs_grid_ids = pd.read_csv(
                    os.path.join(swatmf_model, 'modflow.obs'),
                    sep=r'\s+',
                    usecols=[3, 4],
                    skiprows=2,
                    header=None
                    )

In [22]:
mf_sim = pd.read_csv(
                     os.path.join(swatmf_model, 'swatmf_out_MF_obs'), skiprows=1, sep=r'\s+',
                    names=col_names,
                    # usecols=grids,
                    )
mf_sim.index = pd.date_range(sim_start, periods=len(mf_sim))
mf_sim = mf_sim[sim_start:cal_end]

In [23]:
swatmf_pst_utils.extract_depth_to_water(grids, sim_start, cal_end)

dtw_501.txt file has been created...
Finished ...


## 4.4 match it with modflow.obd file (MODFLOW)

In [24]:
swatmf_pst_utils.mf_obd_to_ins('dtw_501.txt', 'g_5699', cal_start, cal_end)

dtw_501.txt.ins file has been created...


date
1985-01-01                        l1
1985-01-02                        l1
1985-01-03                        l1
1985-01-04                        l1
1985-01-05                        l1
                       ...          
1985-12-27    l1 w !g_5699_19851227!
1985-12-28    l1 w !g_5699_19851228!
1985-12-29    l1 w !g_5699_19851229!
1985-12-30    l1 w !g_5699_19851230!
1985-12-31    l1 w !g_5699_19851231!
Freq: D, Name: g_5699_ins, Length: 365, dtype: object

# 5. Create PEST control file

In [25]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'mb_zon_dummy.pst')

noptmax:30, npar_adj:7, nnz_obs:305


In [26]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
cn2,cn2,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
esco,esco,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
hk01,hk01,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
hk02,hk02,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
sol_awc(),sol_awc(),log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
sy01,sy01,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
sy02,sy02,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1


## 5.1 Assign parameter group name

In [27]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    else:
        par.iloc[i, 6] = 'swat'
print(par)

              parnme partrans parchglim  parval1       parlbnd       parubnd  \
cn2              cn2      log    factor      1.0  1.100000e-10  1.100000e+10   
esco            esco      log    factor      1.0  1.100000e-10  1.100000e+10   
hk01            hk01      log    factor      1.0  1.100000e-10  1.100000e+10   
hk02            hk02      log    factor      1.0  1.100000e-10  1.100000e+10   
sol_awc()  sol_awc()      log    factor      1.0  1.100000e-10  1.100000e+10   
sy01            sy01      log    factor      1.0  1.100000e-10  1.100000e+10   
sy02            sy02      log    factor      1.0  1.100000e-10  1.100000e+10   

          pargp  scale  offset  dercom  
cn2        swat    1.0     0.0       1  
esco       swat    1.0     0.0       1  
hk01         hk    1.0     0.0       1  
hk02         hk    1.0     0.0       1  
sol_awc()  swat    1.0     0.0       1  
sy01         sy    1.0     0.0       1  
sy02         sy    1.0     0.0       1  


## 5.2 Adjust initial parameter values and their ranges

In [28]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 1  
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 1.000000e-02       
        par.iloc[i, 4] = 1.000000e-04
        par.iloc[i, 5] = 0.6  
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

3

In [29]:
# CN2
par.loc['cn2', 'parval1'] = 1.001
par.loc['cn2', 'parlbnd'] = 0.8
par.loc['cn2', 'parubnd'] = 1.2
par.loc['cn2', 'offset'] = -1

# ESCO
par.loc['esco', 'parval1'] = 1.001
par.loc['esco', 'parlbnd'] = 0.5
par.loc['esco', 'parubnd'] = 1.5
par.loc['esco', 'offset'] = -1

# sol_awc()
par.loc['sol_awc()', 'parval1'] = 1.001
par.loc['sol_awc()', 'parlbnd'] = 0.5
par.loc['sol_awc()', 'parubnd'] = 1.5
par.loc['sol_awc()', 'offset'] = -1


## 5.3 Assign parameter group name

In [30]:
# set observation group
obd = pst.observation_data
obd

,obsnme,obsval,weight,obgnme
g_5699_19851130,g_5699_19851130,-0.80200,1.0,obgnme
g_5699_19851201,g_5699_19851201,-0.82500,1.0,obgnme
g_5699_19851202,g_5699_19851202,-0.84400,1.0,obgnme
g_5699_19851203,g_5699_19851203,-0.85100,1.0,obgnme
g_5699_19851204,g_5699_19851204,-0.85700,1.0,obgnme
...,...,...,...,...
rch058_19850926,rch058_19850926,0.23810,1.0,obgnme
rch058_19850927,rch058_19850927,0.04602,1.0,obgnme
rch058_19850928,rch058_19850928,0.43360,1.0,obgnme
rch058_19850929,rch058_19850929,0.46530,1.0,obgnme


In [31]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

,obsnme,obsval,weight,obgnme
g_5699_19851130,g_5699_19851130,-0.80200,1.0,g_5699
g_5699_19851201,g_5699_19851201,-0.82500,1.0,g_5699
g_5699_19851202,g_5699_19851202,-0.84400,1.0,g_5699
g_5699_19851203,g_5699_19851203,-0.85100,1.0,g_5699
g_5699_19851204,g_5699_19851204,-0.85700,1.0,g_5699
...,...,...,...,...
rch058_19850926,rch058_19850926,0.23810,1.0,rch058
rch058_19850927,rch058_19850927,0.04602,1.0,rch058
rch058_19850928,rch058_19850928,0.43360,1.0,rch058
rch058_19850929,rch058_19850929,0.46530,1.0,rch058


## 5.4 Provide actual observed values to control file

In [32]:
os.getcwd()

'D:\\test\\middle_bosque_1000\\SWAT-MODFLOW'

In [33]:
# Streamflow
stf_obd = pd.read_csv('swat_rch_day.obd',
                       sep=r'\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

,rch058
date,
1985-01-01,7.957021
1985-01-02,5.804944
1985-01-03,5.380192
1985-01-04,5.238608
1985-01-05,4.530688
...,...
1985-09-26,0.018689
1985-09-27,0.018689
1985-09-28,0.025202


In [34]:
# watertable
dtw_obd = pd.read_csv('dtw_day.obd',
                       sep=r'\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
dtw_obd = dtw_obd[cal_start: cal_end]
dtw_obd

,g_5699,g_5832
date,,
1985-11-27,NaN,-2.758
1985-11-28,NaN,-2.835
1985-11-29,NaN,-2.850
1985-11-30,-0.777,-2.880
1985-12-01,-0.823,-2.850
1985-12-02,-0.838,-2.835
1985-12-03,-0.838,-2.819
1985-12-04,-0.838,-2.813
1985-12-05,-0.838,-2.804


In [35]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

['g_5699', 'rch058']

In [36]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

g_5699
rch058


305

In [37]:
obd.loc[:, 'obsval'] = tot_obd
obd

,obsnme,obsval,weight,obgnme
g_5699_19851130,g_5699_19851130,-0.777000,1.0,g_5699
g_5699_19851201,g_5699_19851201,-0.823000,1.0,g_5699
g_5699_19851202,g_5699_19851202,-0.838000,1.0,g_5699
g_5699_19851203,g_5699_19851203,-0.838000,1.0,g_5699
g_5699_19851204,g_5699_19851204,-0.838000,1.0,g_5699
...,...,...,...,...
rch058_19850926,rch058_19850926,0.018689,1.0,rch058
rch058_19850927,rch058_19850927,0.018689,1.0,rch058
rch058_19850928,rch058_19850928,0.025202,1.0,rch058
rch058_19850929,rch058_19850929,4.898806,1.0,rch058


# 6. Create new control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [38]:
pst.control_data.formatted_values

name
rstfle                        restart
pestmode                   estimation
npar                                0
nobs                                0
npargp                              0
nprior                              0
nobsgp                              0
maxcompdim                          0
ntplfle                             0
ninsfle                             0
precis                         single
dpoint                          point
numcom                              1
jacfile                             0
messfile                            0
obsreref                   noobsreref
rlambda1                 2.000000E+01
rlamfac                 -3.000000E+00
phiratsuf                3.000000E-01
phiredlam                1.000000E-02
numlam                             -7
jacupdate                         999
lamforgive                 lamforgive
derforgive               noderforgive
relparmax                1.000000E+01
facparmax                1.000000E+01
facorig

In [39]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

### add new PEST++ variables like so:

In [40]:
# A few examples of adding PEST++ options of different types:
# pestpp-ies; the number of realizations to draw in order to form parameter and observation ensembles.
pst.pestpp_options['ies_num_reals'] = 50

# specifies a list of values for the Marquardt lambda used in calculation of parameter upgrades. 
# pst.pestpp_options["lambdas"] = [0.1, 1, 10, 100, 1000]
# pst.pestpp_options["ies_bad_phi_sigma"] = 2.0

# pestpp-da; True/False, specify whether to use the simulated states at the end of each cycle as the initial states for the next cycle.   
# pst.pestpp_options['da_use_simulated_states'] = True

# check the dictionary again
pst.pestpp_options

{'ies_num_reals': 50}

In [41]:
pst.write('mb_zon.pst', version=2)

noptmax:0, npar_adj:7, nnz_obs:305


You can also read and load the existing pest control file.

In [56]:
pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [57]:
pst_read.parameter_data

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
parnme,,,,,,,,,,
cn2,cn2,log,factor,1.001,0.8000,1.2,swat,1.0,-1.0,1
esco,esco,log,factor,1.001,0.5000,1.5,swat,1.0,-1.0,1
hk01,hk01,log,factor,1.000,0.0100,100.0,hk,1.0,0.0,1
hk02,hk02,log,factor,1.000,0.0100,100.0,hk,1.0,0.0,1
sol_awc(),sol_awc(),log,factor,1.001,0.5000,1.5,swat,1.0,-1.0,1
sy01,sy01,log,factor,0.010,0.0001,0.6,sy,1.0,0.0,1
sy02,sy02,log,factor,0.010,0.0001,0.6,sy,1.0,0.0,1
